In [1]:
import os
from dotenv import load_dotenv

from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser




In [3]:
load_dotenv()

os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]="rag_test0731"

# 문서 로드

In [4]:
loader = TextLoader("./folktales/춘향전1편.txt")
documents = loader.load()
documents # list

[Document(metadata={'source': './folktales/춘향전1편.txt'}, page_content='緣分(연분)\n『여바라 방자야!』\n\n하고 책상 위에 펴 놓은 책도 보는 듯 마는 듯 우두커니 하고 무엇을 생각하고 앉았던 몽룡(夢龍)은 소리를 치었다.\n\n『여이.』\n\n하고 익살덩어리로 생긴 방자가 어깨짓을 하고 뛰어 들어 와 책방 층계 앞에 읍하고 선다.\n\n몽룡은 책상 위에 들어오는 볕을 막노라고 반쯤 닫히었던 영창을 성가신 듯이 와락 밀며,\n\n『얘, 너의 남원 고을에 어디 볼 만한 것이 없느냐?』\n\n방자는 의외에 말을 듣는 듯이 고개를 숙인 대로 눈을 치 떠서 물끄러미 몽룡을 치어다보더니,\n\n『소인의 골엔들 어찌 볼 만한 곳이 없을 리가 있읍니까.\n\n산으로 가오면 나물 캐는 것도 볼 만하옵고, 들로 가오면 농사짓는 것도 볼 만하옵고, 우물로 나가오면 여편네들 물 길어 놓고 밥솥에 밥 눗는 것도 다 잊어버리고 수다 늘어 놓는 것도 볼 만하옵고, 또 행길로 나가오면 술주정군이 술 주정하는 것, 술취한 남편 붙들고 내외 싸움하는 것도 볼 만하옵고......』\n\n『에라 이놈아!』\n\n하고 몽룡은 괘씸한 듯이 책상을 딱 치며, 누가 그런 소리 너더러 줏어대라드냐. 어디 경치 볼만한 곳이 있느냐 말이다—어 그놈.\n\n『네? 그렇거든 애시에 그렇게 말씀하실 께지 소인인들 힘 들여 번 밥 먹은 기운을 헛소리에 다 써버리고 싶을 리가 있겠읍니까...... 소인의 골에 경치 볼 만한 곳으로 말씀하오 면 북문 밖에 조종산성 좋다 하옵고 서문 밖에 관왕묘도 그 럴 듯하다 하오나 제일 이름이 높기로는 남문 밖 나서서 광 한루와 오작교온데 경개 절승하옵니다. 과시 삼남에 제일 명승지라 할 만하옵지요.』\n\n『광한루라 광한루, 오작교 오작교.』\n\n하고 몽룡은 혼자 입속으로 불러 보더니,\n\n『얘, 광한루 오작교 이름이 좋다—광한루로 나가자. 나귀 안장 지어라.』\n\n이 말에 크게 놀라는 듯이 방자가 껑충 뛰며,\n\n

# 문서 분할

In [5]:
text_splitter = CharacterTextSplitter(
    # 텍스트를 분할할 때 사용할 구분자를 지정합니다. 기본값은 "\n\n"입니다.
    # separator=" ",
    # 분할된 텍스트 청크의 최대 크기를 지정합니다.
    chunk_size=1000,
    # 분할된 텍스트 청크 간의 중복되는 문자 수를 지정합니다.
    chunk_overlap=100,
    # 텍스트의 길이를 계산하는 함수를 지정합니다.
    length_function=len,
    # 구분자가 정규식인지 여부를 지정합니다.
    is_separator_regex=False,
)

In [6]:
split_documents = text_splitter.split_documents(documents)
split_documents

[Document(metadata={'source': './folktales/춘향전1편.txt'}, page_content='緣分(연분)\n『여바라 방자야!』\n\n하고 책상 위에 펴 놓은 책도 보는 듯 마는 듯 우두커니 하고 무엇을 생각하고 앉았던 몽룡(夢龍)은 소리를 치었다.\n\n『여이.』\n\n하고 익살덩어리로 생긴 방자가 어깨짓을 하고 뛰어 들어 와 책방 층계 앞에 읍하고 선다.\n\n몽룡은 책상 위에 들어오는 볕을 막노라고 반쯤 닫히었던 영창을 성가신 듯이 와락 밀며,\n\n『얘, 너의 남원 고을에 어디 볼 만한 것이 없느냐?』\n\n방자는 의외에 말을 듣는 듯이 고개를 숙인 대로 눈을 치 떠서 물끄러미 몽룡을 치어다보더니,\n\n『소인의 골엔들 어찌 볼 만한 곳이 없을 리가 있읍니까.\n\n산으로 가오면 나물 캐는 것도 볼 만하옵고, 들로 가오면 농사짓는 것도 볼 만하옵고, 우물로 나가오면 여편네들 물 길어 놓고 밥솥에 밥 눗는 것도 다 잊어버리고 수다 늘어 놓는 것도 볼 만하옵고, 또 행길로 나가오면 술주정군이 술 주정하는 것, 술취한 남편 붙들고 내외 싸움하는 것도 볼 만하옵고......』\n\n『에라 이놈아!』\n\n하고 몽룡은 괘씸한 듯이 책상을 딱 치며, 누가 그런 소리 너더러 줏어대라드냐. 어디 경치 볼만한 곳이 있느냐 말이다—어 그놈.\n\n『네? 그렇거든 애시에 그렇게 말씀하실 께지 소인인들 힘 들여 번 밥 먹은 기운을 헛소리에 다 써버리고 싶을 리가 있겠읍니까...... 소인의 골에 경치 볼 만한 곳으로 말씀하오 면 북문 밖에 조종산성 좋다 하옵고 서문 밖에 관왕묘도 그 럴 듯하다 하오나 제일 이름이 높기로는 남문 밖 나서서 광 한루와 오작교온데 경개 절승하옵니다. 과시 삼남에 제일 명승지라 할 만하옵지요.』\n\n『광한루라 광한루, 오작교 오작교.』\n\n하고 몽룡은 혼자 입속으로 불러 보더니,\n\n『얘, 광한루 오작교 이름이 좋다—광한루로 나가자. 나귀 안장 지어라.』\n\n이 말에 크게 놀라는 듯이 방자가 껑충 뛰며,'),


In [7]:
print(f"분할된 청크의수: {len(split_documents)}")

분할된 청크의수: 25


# 임베딩 모델 생성

In [8]:
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")
embeddings

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x7f1c9d092190>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x7f1c9a7e8a50>, model='text-embedding-3-large', dimensions=None, deployment='text-embedding-ada-002', openai_api_version='', openai_api_base=None, openai_api_type='', openai_proxy='', embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

# DB 생성 및 저장

In [9]:
vectorstore = Chroma.from_documents(documents=split_documents, embedding=embeddings, collection_name="chunhyang1")
vectorstore

# 검색기 생성

In [10]:
retriever = vectorstore.as_retriever()
retriever

VectorStoreRetriever(tags=['Chroma', 'OpenAIEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7f1c9a757950>)

In [11]:
retriever.invoke("춘향의 성격을 잘 드러내는 문서는?")

[Document(metadata={'source': './folktales/춘향전1편.txt'}, page_content='하고 옷고름을 끌러 옷자락으로 부채질을 하며 짖궂게,\n\n『참 술맛 좋읍디다.』\n\n『그래 술 얻어 먹고 그리고는 어쨌어?』\n\n하고 몽룡은 심히 맘이 조급하였다.\n\n『술 먹고는 안주 먹었지요. 문어 발 먹었지요. 그놈 질깁 디다.』\n\n몽룡이 견디다 못하여,\n\n『이놈아, 술 먹고 안주 먹고 그것뿐이야? 편지는 어찌했 단 말이냐?』\n\n방자 능청스럽게 놀라는 듯이 한 번 껑충 뛰고 머리를 긁 적긁적하며,\n\n『아차, 술맛이 하도 좋킬래 길 오면서 술 생각만 하노라 고 도련님 편지는 미처 생각도 못하였소...... 편지는 갖다 주 었지요.』\n\n『그래서?』\n\n하고 몽룡의 기색이 좀 풀린다.\n\n『춘향이가 읽어요.』\n\n『그리고는?』\n\n『또 한번 읽어요.』\n\n『그리고는?』\n\n『또 읽던가 보든 걸요.』\n\n『이놈아, 춘향 아씨가 편지를 읽고는 어떻게 하더냐 말이 야...... 허 그놈 사람의 애를 식은 재가 되도록 다 태워버리 고야 말려는구나.』\n\n『소인의 애는 얼마나 탔는 데요?』\n\n『그래 편지를 읽고는?』\n\n『자세히 말씀해요?』\n\n『그래 자세히 말해라.』\n\n방자 잊었던 것을 생각한느 모양으로 한참이나 고개를 기 웃기웃하더니,\n\n『춘향이가 도련님 편지를 읽고는—아마 열 일곱 번은 읽나 봅디다. 한참은 몇 번이나 읽나 보자 하고 세이다가 열 댓 까지 세이고는 구찮아서 말았소.』\n\n『압다, 이놈아 그래 편지를 읽고는 어찌하더냐 말이야?』\n\n하고 몽룡이 갑갑증이 나서 발로 마루를 한 번 구른다.\n\n방자는 놀라는 듯 두려워하는 듯 또 한 번 껑충 뛰며,\n\n『네 바로 아뢰오리다—춘향이가 그 편지를 읽더니마는—아 마 열 일곱 번이나 읽더니마는 두 빰은 발그레 두 입술은 오물오물 두 눈은 사르르 숨소리는 쌔근쌔근하더니만 제 어 미 월매를 보고 「답장 써요?」하옵디다.』\n\n하고

# 프롬프트 생성

In [12]:
prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

# 언어 모델 생성

In [13]:
llm = ChatOpenAI(model="gpt-4o-mini")
llm

ChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x7f1c9944a8d0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x7f1c99457d10>, model_name='gpt-4o-mini', openai_api_key=SecretStr('**********'), openai_proxy='')

# 체인 생성

In [14]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [15]:
question = "삼성전자가 자체 개발한 AI 의 이름은?"
response = chain.invoke(question)
print(response)

모릅니다.


In [16]:
question = "춘향이는 어떤 성격이야?"
response = chain.invoke(question)
print(response)

춘향이는 매우 똑똑하고 자존심이 강한 성격을 가지고 있습니다. 그녀는 자신의 행실에 대해 강한 주관을 가지고 있으며, 방자의 말에 반박하며 자신의 입장을 분명히 합니다. 또한, 그녀는 예의와 체면을 중요시하며, 모르는 남자의 전갈을 듣고 따라가는 것을 거부하는 등 신중한 면모도 보여줍니다. 이러한 모습은 그녀가 단순히 순종적인 성격이 아니라, 강한 의지를 가진 인물임을 나타냅니다.


In [17]:
question = "이 이야기의 핵심 사건은 뭐야?"
response = chain.invoke(question)
print(response)

이 이야기의 핵심 사건은 몽룡이 광한루에서 그네를 뛰는 춘향을 보고 매혹되어 그녀에게 관심을 갖게 되는 장면입니다. 몽룡은 춘향의 아름다움에 감동하여 그녀를 바라보며 가슴이 두근거리는 감정을 느끼고, 결국 춘향에게 다가가게 되는 상황이 전개됩니다.
